# **Problemas cardiológicos selecionados**

O modelo será treinado para reconhecer três condições distintas nos registros de ECG da base PTB-XL:

1. Infarto Agudo do Miocárdio (MI)

  - Condição grave com alterações claras na morfologia do complexo QRS e segmento ST.

2. Batimento Cardíaco Anormal (Abnormal Heartbeat – Abn-HB)

  - Irregularidades no ritmo cardíaco, incluindo extrassístoles, fibrilações e taquiarritmias.

3. ECG Normal (NORM)

  - Representa pacientes saudáveis, sem evidências de anomalias eletrocardiográficas.

A inclusão da classe “normal” é essencial para permitir que o modelo aprenda a distinguir padrões fisiológicos normais de padrões patológicos. Isso garante completude diagnóstica e redução de falsos positivos, além de estar em conformidade com a especificação oficial do projeto: “diagnóstico de anomalias cardiológicas e também coração normal”.

---
# **Pré-processamento**

## Instalando bibliotecas necessárias:

In [2]:
pip install wfdb pandas numpy scipy matplotlib scikit-learn seaborn PyWavelets tqdm


Defaulting to user installation because normal site-packages is not writeable
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 1.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


---
## Processamento da Base PTB-XL

### Tratamento dos dados 

Essa primeira etapa realiza a importação, organização, balanceamento e salvamento dos dados do dataset [PTB-XL](https://physionet.org/content/ptb-xl/1.0.3/), uma base pública de eletrocardiogramas, incluindo exames com infarto (MI), batimentos anormais (Abn-HB) e normalidade (NORM)

- Feitos do código abaixo:
    1. Importa os metadados do arquivo ptbxl_database.csv
    2. Classifica os exames em três categorias clínicas:
    - MI: Infarto do miocárdio (com base nos códigos SCP)
    - Abn-HB: Batimentos anormais (como PVC, LBBB, PAC)
    - NORM: Eletrocardiogramas normais
    3. Seleciona uma amostra balanceada com até 100 exames por classe
    4. Carrega os sinais de ECG brutos (500 Hz) usando o wfdb
    5. Salva os sinais e rótulos como arquivos .npy em data_preprocessed/

- Saídas:
    1. `data_preprocessed/X_signals.npy`: Lista de sinais ECG brutos (array de objetos NumPy)
    2. `data_preprocessed/y_labels.npy`: Lista de rótulos (MI, Abn-HB, NORM)

In [3]:
# Bibliotecas necessárias
import os
import pandas as pd
import numpy as np
import wfdb            # para carregar os sinais .dat/.hea
import ast             # para converter string → dicionário (scp_codes)

# Caminhos principais do projeto
BASE_DIR = os.path.expanduser('~/projetos/ptbxl')                   # diretório base do projeto
RECORDS_PATH = os.path.join(BASE_DIR, 'records500')                 # onde estão os sinais ECG
METADATA_CSV = os.path.join(BASE_DIR, 'ptbxl_database.csv')         # metadados dos exames
SCP_STATEMENTS_CSV = os.path.join(BASE_DIR, 'scp_statements.csv')   # mapeia códigos SCP
DATA_OUT_PATH = os.path.join(BASE_DIR, 'data_preprocessed')         # onde salvar o dataset pronto

# Criação da pasta de saída 
os.makedirs(DATA_OUT_PATH, exist_ok=True)

# Carrega o CSV de metadados e converte scp_codes de string para dict
def load_metadata():
    df = pd.read_csv(METADATA_CSV)
    df['scp_codes'] = df['scp_codes'].apply(ast.literal_eval)
    return df

# Mapeia os códigos SCP para três classes clínicas: MI, Abn-HB, NORM
def extract_labels(df):
    
    # Lê os significados dos códigos SCP e filtra os que são infarto (MI)
    scp_df = pd.read_csv(SCP_STATEMENTS_CSV, index_col=0)
    mi_scp_codes = scp_df[scp_df['diagnostic_class'] == 'MI'].index.tolist()

    # Define os códigos de batimentos anormais
    abn_scp_codes = ['ABQRS', 'PVC', 'PAC', 'LBBB', 'RBBB', 'IRBBB']

    # Função auxiliar para atribuir rótulo com base nos códigos presentes
    def map_label(codes):
        labels = list(codes.keys())
        if any(label in mi_scp_codes for label in labels):
            return 'MI'
        elif any(label in abn_scp_codes for label in labels):
            return 'Abn-HB'
        elif 'NORM' in labels:
            return 'NORM'
        else:
            return None

    # Aplica o mapeamento e remove exames sem classe definida
    df['target'] = df['scp_codes'].apply(map_label)
    return df[df['target'].notnull()]

# Carrega o sinal de ECG a partir de um caminho relativo (filename_hr)
def load_signal(record_path):
    full_path = os.path.join(BASE_DIR, record_path)
    signal, _ = wfdb.rdsamp(full_path)
    return signal

# Carrega um subconjunto balanceado de sinais com as três classes
def load_dataset_balanced(sample_limit_per_class=100):
    df = extract_labels(load_metadata())

    dfs = []
    for class_name in ['MI', 'Abn-HB', 'NORM']:
        df_class = df[df['target'] == class_name]
        available = len(df_class)
        if available == 0:
            print(f"Classe '{class_name}' indisponível na base.")
            continue
        n = min(sample_limit_per_class, available)
        print(f"Classe '{class_name}': usando {n} de {available} disponíveis.")
        dfs.append(df_class.sample(n=n, random_state=42))  # amostragem aleatória

    # Une os subconjuntos e embaralha os dados
    df_sampled = pd.concat(dfs).sample(frac=1, random_state=42)

    X_signals = []  # lista de sinais ECG
    y_labels = []   # lista de rótulos

    print(f"\nCarregando {len(df_sampled)} sinais balanceados...")

    # Carrega os sinais reais dos arquivos .dat/.hea
    for _, row in df_sampled.iterrows():
        try:
            signal = load_signal(row['filename_hr'])
            X_signals.append(signal)
            y_labels.append(row['target'])
        except Exception as e:
            print(f"Erro ao carregar {row['filename_hr']}: {e}")

    return np.array(X_signals, dtype=object), np.array(y_labels)

# Salva os sinais e rótulos em arquivos .npy
def save_dataset(X, y):
    np.save(os.path.join(DATA_OUT_PATH, 'X_signals.npy'), X)
    np.save(os.path.join(DATA_OUT_PATH, 'y_labels.npy'), y)
    print(f"\nDados salvos em: {DATA_OUT_PATH}")
    print(f"   - {len(X)} sinais")
    print(f"   - Distribuição: {dict(zip(*np.unique(y, return_counts=True)))}")

# Execução principal (para analisar a distribuição)
if __name__ == "__main__":
    X, y = load_dataset_balanced(sample_limit_per_class=100)
    save_dataset(X, y)

    # Exibe a distribuição geral das classes no conjunto total
    dff = load_metadata()
    dff = extract_labels(dff)
    print("\nDistribuição de classes (conjunto total):")
    print(dff['target'].value_counts())


Classe 'MI': usando 100 de 5469 disponíveis.
Classe 'Abn-HB': usando 100 de 2222 disponíveis.
Classe 'NORM': usando 100 de 8748 disponíveis.

Carregando 300 sinais balanceados...

Dados salvos em: /home/vinicius/projetos/ptbxl/data_preprocessed
   - 300 sinais
   - Distribuição: {np.str_('Abn-HB'): np.int64(100), np.str_('MI'): np.int64(100), np.str_('NORM'): np.int64(100)}

Distribuição de classes (conjunto total):
target
NORM      8748
MI        5469
Abn-HB    2222
Name: count, dtype: int64


---
### Visualização dos arquivos gerados

#### Arquivo X_signals

In [4]:
X = np.load('data_preprocessed/X_signals.npy', allow_pickle=True)

print(type(X))                 # <class 'numpy.ndarray'>
print(X.shape)                 # (300,)  ← 300 exames (cada um é um array 2D)
print(X[0].shape)              # (5000, 12)  ← 5000 amostras, 12 derivações
print(X[0])                    # mostra o sinal do primeiro exame


<class 'numpy.ndarray'>
(300, 5000, 12)
(5000, 12)
[[-0.095 -0.01 0.085 ... 0.105 -0.115 -0.05]
 [-0.095 -0.01 0.085 ... 0.105 -0.115 -0.05]
 [-0.095 -0.01 0.085 ... 0.105 -0.115 -0.05]
 ...
 [0.26 0.18 -0.08 ... 0.695 0.46 0.395]
 [0.26 0.18 -0.08 ... 0.695 0.46 0.395]
 [0.26 0.18 -0.08 ... 0.695 0.46 0.395]]


- Formato: cada elemento do array é um sinal ECG de um exame, carregado a partir dos arquivos .dat e .hea.
- Dimensão de cada item: (n_amostras, n_derivações), geralmente (5000, 12) para 10 segundos de ECG a 500 Hz com 12 derivações.
- Tamanho do array: 100 por classe (3 classes: MI, Abn-HB, NORM), então o total é 300 sinais

---
#### Arquivo y_signals 

In [5]:
y = np.load('data_preprocessed/y_labels.npy', allow_pickle=True)

print(type(y))         # <class 'numpy.ndarray'>
print(y.shape)         # (300,)
print(np.unique(y))    # ['Abn-HB' 'MI' 'NORM']
print(y[0])            # rótulo do primeiro sinal em X

<class 'numpy.ndarray'>
(300,)
['Abn-HB' 'MI' 'NORM']
NORM


- Tipo: numpy.ndarray de strings
- Formato: vetor unidimensional com o rótulo correspondente a cada sinal em X_signals.npy.
- Valores possíveis: 'MI', 'Abn-HB', 'NORM'